In [ ]:
import zipfile
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.model_selection import train_test_split

In [ ]:
# Unzip the dataset
with zipfile.ZipFile('Violence Detection.v1-v1.multiclass.zip', 'r') as zip_ref:
    zip_ref.extractall('multiclass_dataset')

In [ ]:
# Load your CSV dataset
train_csv_path = 'multiclass_dataset/train/_classes.csv'
valid_csv_path = 'multiclass_dataset/valid/_classes.csv'
test_csv_path = 'multiclass_dataset/test/_classes.csv'

In [ ]:
train_data = pd.read_csv(train_csv_path)
valid_data = pd.read_csv(valid_csv_path)
test_data = pd.read_csv(test_csv_path)

In [ ]:
# Preprocess the image data
image_data = []
image_labels = []

In [ ]:
image_directory = 'multiclass_dataset/train/'  # Adjust to the appropriate folder

In [ ]:
print(train_data.columns)

Index(['filename', ' NonViolence', ' Violence'], dtype='object')


In [ ]:
for index, row in train_data.iterrows():
    filename = row['filename']
    label_nonViolence = row[' NonViolence']  # Note the leading space
    label_Violence = row[' Violence']  # Note the leading space

    image_path = os.path.join(image_directory, filename)
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = preprocess_input(image)

    image_data.append(image)
    image_labels.append([label_nonViolence, label_Violence])

In [ ]:
image_data = np.array(image_data)
image_labels = np.array(image_labels)

In [ ]:
# Load the ResNet-50 model pre-trained on ImageNet
image_input = Input(shape=(224, 224, 3))
resnet_base = ResNet50(weights='imagenet', include_top=False, input_tensor=image_input)

In [ ]:
# Combine image data and text data (sequences) if needed

# Build a classification head
x = Flatten()(resnet_base.output)
x = Dense(128, activation='relu')(x)
num_classes = 2  # Two classes: nonViolence and Violence
output = Dense(num_classes, activation='softmax')(x)

In [ ]:
# Create the hybrid model
model = Model(inputs=image_input, outputs=output)

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Split the dataset into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(image_data, image_labels, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
# Train the model
batch_size = 64
epochs = 10
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val))

Epoch 1/10
30/30 [==============================] - 56s 629ms/step - loss: 13.1776 - accuracy: 0.5356 - val_loss: 150867.6250 - val_accuracy: 0.5342
Epoch 2/10
30/30 [==============================] - 18s 596ms/step - loss: 13.4073 - accuracy: 0.5409 - val_loss: 983730.3750 - val_accuracy: 0.4658
Epoch 3/10
30/30 [==============================] - 19s 623ms/step - loss: 7.2328 - accuracy: 0.5645 - val_loss: 15865.8574 - val_accuracy: 0.5427
Epoch 4/10
30/30 [==============================] - 18s 613ms/step - loss: 23.7117 - accuracy: 0.5570 - val_loss: 11840.0947 - val_accuracy: 0.4615
Epoch 5/10
30/30 [==============================] - 19s 639ms/step - loss: 63.2047 - accuracy: 0.5581 - val_loss: 1050667.8750 - val_accuracy: 0.5342
Epoch 6/10
30/30 [==============================] - 19s 625ms/step - loss: 93.0837 - accuracy: 0.5490 - val_loss: 1073367.8750 - val_accuracy: 0.5342
Epoch 7/10
30/30 [==============================] - 19s 617ms/step - loss: 109.8684 - accuracy: 0.5655 - va

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

8/8 [==============================] - 1s 92ms/step - loss: 4655.0845 - accuracy: 0.5000
Test Accuracy: 50.00%
